In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from mediapipe.python.solutions import holistic as mp_holistic

In [ ]:
#load original skeleton / center_of_mass data
session_path = Path(r"D:\Dropbox\FreeMoCapProject\FreeMocap_Data\sesh_2022-11-09_20_38_23_neurons_demo")
output_data_path = session_path / "DataArrays"
og_skeleton_npy_file_name = "mediapipe_body_3d_xyz.npy"
og_skeleton_npy_path = output_data_path / og_skeleton_npy_file_name

original_skeleton_frame_marker_xyz = np.load(og_skeleton_npy_path) / 1000 #convert to meters (b/c that's Blender's unit, which is where we got the transformation matrix)
number_of_frames = original_skeleton_frame_marker_xyz.shape[0]
number_of_markers = original_skeleton_frame_marker_xyz.shape[1]

# load center_of_mass data
og_center_of_mass_npy_file_name = "totalBodyCOM_frame_XYZ.npy"
original_center_of_mass_xyz = np.load(output_data_path / og_center_of_mass_npy_file_name) / 1000

In [ ]:
#to get this data,
# -  open `.blend` file (or run `src/blender_stuff/load_npy_as_empties.py`),
# -  rotate skeleton until it looks good, then ... 
# -  enter `bpy.data.objects['freemocap_origin_axes'].matrix_world` into console
# -  copy the output into the `transformation_matrix` variable below (changing ()'s to []'s)

transformation_matrix = \
[[0.8891741633415222, -0.4202311933040619, -0.181038960814476, 0.07594677060842514],
[0.40811246633529663, 0.9072690606117249, -0.10152354836463928, 0.0],
[0.20691442489624023, 0.016387859359383583, 0.9782217741012573, 0.14920693635940552],
[0.0, 0.0, 0.0, 1.0]]

In [ ]:
transformed_skeleton_frame_marker_xyz = np.zeros_like(original_skeleton_frame_marker_xyz)
transformed_center_of_mass_frame_xyz = np.zeros((number_of_frames, 3))

for frame_number in range(number_of_frames):
    
    transformed_com_xyzw = transformation_matrix @ np.append(original_center_of_mass_xyz[frame_number], 1)
    transformed_center_of_mass_frame_xyz[frame_number] = transformed_com_xyzw[:3]

    for marker_number in range(number_of_markers):
        point_xyz = original_skeleton_frame_marker_xyz[frame_number, marker_number, :]
        point_xyzw = np.append(point_xyz, 1)
        transformed_point_xyzw = transformation_matrix @ point_xyzw
        transformed_point_xyz = transformed_point_xyzw[:3]
        transformed_skeleton_frame_marker_xyz[frame_number, marker_number, : ] = transformed_point_xyz

print(f"transformed_skeleton_frame_marker_xyz.shape: {transformed_skeleton_frame_marker_xyz.shape}")

#convert back to mm
original_skeleton_frame_marker_xyz *= 1000
original_center_of_mass_xyz *= 1000

transformed_skeleton_frame_marker_xyz*= 1000
transformed_center_of_mass_frame_xyz *= 1000


np.save(output_data_path / "mediapipe_body_3d_xyz_transformed.npy", transformed_skeleton_frame_marker_xyz)

# Now we test if it works by plotting COM projection onto XZ (ground) plane relative to the base of support (i.e. the right toe and heel markers)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# get_relevant marker data out
mediapipe_body_landmark_names = [
    landmark.name.lower() for landmark in mp_holistic.PoseLandmark
]

# original data
original_right_heel_x = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_heel"),0]
original_right_heel_y = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_heel"),1]
original_right_heel_z = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_heel"),2]

original_right_toe_x = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_foot_index"),0]
original_right_toe_y = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_foot_index"),1]
original_right_toe_z = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_foot_index"),2]

original_left_heel_x = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_heel"),0]
original_left_heel_y = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_heel"),1]
original_left_heel_z = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_heel"),2]

original_left_toe_x = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_foot_index"),0]
original_left_toe_y = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_foot_index"),1]
original_left_toe_z = original_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_foot_index"),2]


# transformed data
transformed_right_heel_x = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_heel"),0]
transformed_right_heel_y = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_heel"),1]
transformed_right_heel_z = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_heel"),2]

transformed_right_toe_x = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_foot_index"),0]
transformed_right_toe_y = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_foot_index"),1]
transformed_right_toe_z = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("right_foot_index"),2]

transformed_left_heel_x = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_heel"),0]
transformed_left_heel_y = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_heel"),1]
transformed_left_heel_z = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_heel"),2]

transformed_left_toe_x = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_foot_index"),0]
transformed_left_toe_y = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_foot_index"),1]
transformed_left_toe_z = transformed_skeleton_frame_marker_xyz[:, mediapipe_body_landmark_names.index("left_foot_index"),2]


In [ ]:
# check via bos/com
#identify relevant frames
standing_on_right_foot_frame_start = 431 #watched the output `mp4` for frame numbers
standing_on_right_foot_frame_end = 470
frames_of_interest = np.arange(standing_on_right_foot_frame_start, standing_on_right_foot_frame_end)


In [ ]:

def normalize_axes_2d(axis):
    x_range  = axis.get_xlim()[1] - axis.get_xlim()[0]
    y_range  = axis.get_ylim()[1] - axis.get_ylim()[0]
    
    if x_range > y_range:
        axis.set_ylim(axis.get_ylim()[0], axis.get_ylim()[0] + x_range)
    else:
        axis.set_xlim(axis.get_xlim()[0], axis.get_xlim()[0] + y_range)
    return axis

figure = plt.figure(figsize=(10,10))

## plot original data (Time series)
ax1 = figure.add_subplot(221)
ax1.set_title("Original data - timeseries")
ax1.plot(original_right_heel_x, label="original right heel x")
ax1.plot(original_right_heel_y, label="original right heel y")
ax1.plot(original_right_heel_z, label="original right heel z")
ax1.legend()


## plot original data (on XZ (ground) plane)
ax2 = figure.add_subplot(222)
ax2.set_title("Original data - center of mass vs base of support")
ax2.plot(original_right_toe_x[frames_of_interest],  original_right_toe_y[frames_of_interest], '.-', label="original right toe", color="darkred")
ax2.plot(original_right_heel_x[frames_of_interest], original_right_heel_y[frames_of_interest], 'o-', label="original right heel", color="darkred")

ax2.plot(original_left_toe_x[frames_of_interest],   original_left_toe_y[frames_of_interest],  '.-', label="original left toe", color="darkblue")
ax2.plot(original_left_heel_x[frames_of_interest],  original_left_heel_y[frames_of_interest], 'o-', label="original left heel", color="darkblue")

ax2.plot(com_x[frames_of_interest], com_y[frames_of_interest], 'o-', label="center of mass", color="xkcd:dark sea green")

ax2 = normalize_axes_2d(ax2)
ax2.legend()


## plot transformed data (Time series)
ax3 = figure.add_subplot(223)
ax3.set_title("Transformed data - timeseries")
ax3.plot(transformed_right_heel_x, label="transformed right heel x")
ax3.plot(transformed_right_heel_y, label="transformed right heel y")
ax3.plot(transformed_right_heel_z, label="transformed right heel z")
ax3.legend()

## plot transformed data (on XZ (ground) plane)
ax4 = figure.add_subplot(224)
ax4.set_title("Transformed data - center of mass vs base of support")
ax4.plot(transformed_right_toe_x[frames_of_interest],  transformed_right_toe_y[frames_of_interest], '.-', label="transformed right toe", color="darkred")
ax4.plot(transformed_right_heel_x[frames_of_interest], transformed_right_heel_y[frames_of_interest], 'o-', label="transformed right heel", color="darkred")

ax4.plot(transformed_left_toe_x[frames_of_interest],   transformed_left_toe_y[frames_of_interest],  '.-', label="transformed left toe", color="darkblue")
ax4.plot(transformed_left_heel_x[frames_of_interest],  transformed_left_heel_y[frames_of_interest], 'o-', label="transformed left heel", color="darkblue")

ax4.plot(transformed_center_of_mass_frame_xyz[frames_of_interest,0], transformed_center_of_mass_frame_xyz[frames_of_interest,1], 'o-', label="center of mass", color="xkcd:dark sea green")

ax4 = normalize_axes_2d(ax4)

ax4.legend()


plt.show()